<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/Explorando_dados_do_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

In [0]:
# teste

![alt text](https://drive.google.com/uc/d/1tR4M2maLGBTun3N4yyq-dTzDdQPucFna/view?usp=sharing)